# Feature Engineering

### Importing Data

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('../data/processed_data/cleaned_df.csv')
df['date'] = pd.to_datetime(df['date'])

### Splitting by time

In [2]:
n_days = 90
max_date = df['date'].max()
max_date - pd.to_timedelta(n_days, unit='d')

Timestamp('1998-04-01 00:00:00')

In [3]:
cutoff = max_date - pd.to_timedelta(n_days, unit='d')
temporal_in_df = df[df['date'] <= cutoff]
temporal_out_df = df[df['date'] > cutoff]
print(temporal_in_df['date'].max())
print(temporal_out_df['date'].max())

1998-04-01 00:00:00
1998-06-30 00:00:00


### Engineering Target Features


In [4]:
target_df = temporal_out_df.drop(['quantity', 'date'], axis=1).groupby('customer_id').sum().set_axis(['spend_90_total'], axis=1).reset_index()

In [5]:
target_df['spend_90_flag'] = 1

In [6]:
target_df

,customer_id,spend_90_total,spend_90_flag
0,3,16.99,1
1,9,41.98,1
2,25,73.43,1
3,29,48.45,1
4,31,117.99,1
...,...,...,...
3296,23529,40.56,1
3297,23534,10.48,1
3298,23555,77.44,1
3299,23556,28.98,1


### Engineering RFM Features

In [7]:
temporal_in_max_date = temporal_in_df['date'].max()

In [8]:
recency_feature_df = temporal_in_df[['customer_id', 'date']].groupby('customer_id') \
                      .apply(lambda x: (x['date'].max() - temporal_in_max_date).days) \
                      .to_frame(name='recency').reset_index()
recency_feature_df

,customer_id,recency
0,1,-455
1,2,-444
2,3,-127
3,4,-110
4,5,-88
...,...,...
23565,23566,-372
23566,23567,-372
23567,23568,-344
23568,23569,-372


In [9]:
frequency_feature_df = temporal_in_df[['customer_id', 'date']] \
                        .groupby('customer_id').count().set_axis(['frequency'], axis=1).reset_index()
frequency_feature_df

,customer_id,frequency
0,1,1
1,2,2
2,3,5
3,4,4
4,5,11
...,...,...
23565,23566,1
23566,23567,1
23567,23568,3
23568,23569,1


In [10]:
price_feature_df = temporal_in_df \
                    .groupby('customer_id').agg(
                      {
                        'price':['sum', 'mean']
                      }
                    ).set_axis(['price_sum', 'price_mean'], axis=1).reset_index()
price_feature_df

,customer_id,price_sum,price_mean
0,1,11.77,11.770000
1,2,89.00,44.500000
2,3,139.47,27.894000
3,4,100.50,25.125000
4,5,385.61,35.055455
...,...,...,...
23565,23566,36.00,36.000000
23566,23567,20.97,20.970000
23567,23568,121.70,40.566667
23568,23569,25.74,25.740000


### Merging Features Together

In [11]:
features_df = pd.concat([recency_feature_df, frequency_feature_df, price_feature_df] \
                        , axis=1)
features_df = features_df.loc[:, ~features_df.columns.duplicated()]
features_df = features_df.merge(target_df, how='left', on='customer_id')
features_df = features_df.fillna(0)

In [12]:
features_df

,customer_id,recency,frequency,price_sum,price_mean,spend_90_total,spend_90_flag
0,1,-455,1,11.77,11.770000,0.00,0.0
1,2,-444,2,89.00,44.500000,0.00,0.0
2,3,-127,5,139.47,27.894000,16.99,1.0
3,4,-110,4,100.50,25.125000,0.00,0.0
4,5,-88,11,385.61,35.055455,0.00,0.0
...,...,...,...,...,...,...,...
23565,23566,-372,1,36.00,36.000000,0.00,0.0
23566,23567,-372,1,20.97,20.970000,0.00,0.0
23567,23568,-344,3,121.70,40.566667,0.00,0.0
23568,23569,-372,1,25.74,25.740000,0.00,0.0


### Exporting Data

In [13]:
features_df.to_csv('../data/processed_data/feature_engineered.csv', index=False)